# Survival Models for Lung Cancer Prognosis

This notebook focuses on building and evaluating survival models to predict the survival duration of lung cancer patients based on their 3D CT scans and meta-data.

We will explore three different survival models:

- **Survival Random Forest (RSF)**.
- **Survival Gradient Boosting (sXGB)**.
- **Survival Support Vector Machines (SVM)**.

## Objectives

- Preprocess the data.
- Train and evaluate the models using survival-specific metrics such as the Concordance Index.
- Compare model performances and identify the most effective approach for predicting lung cancer survival outcomes.
- Leverage the CT scans images together with the metadata in order to increase c-index score.

## Dataset Overview

- **Meta-data:** Patient clinical information from `./data`.

---

Let’s start by importing the necessary libraries and loading the data!

In [ ]:
# Preprocessing.
import numpy as np
import pandas as pd
from PIL import Image
import os

# 

In [ ]:
# Import the data.
radiomics = pd.read_csv('radiomics.csv')
clinical = pd.read_csv('clinical_data.csv')
labels = pd.read_csv('labels.csv')